In [16]:
import requests
from bs4 import BeautifulSoup
import re
Register_number = []

# providing url
url = 'https://www.canada.ca/en/environment-climate-change/services/canadian-environmental-protection-act-registry/substances-list/export-control-list.html'
  
# creating request object
page = requests.get(url,verify = "Y:\\Python Notebooks\\Chubb_CA_2016.cer")
soup = BeautifulSoup(page.content, "html.parser")
for para in soup.find_all("li"):
    string = para.get_text()
    if re.search("[0-9]+-[0-9]+-[0-9]+", string) :
        d = re.findall("[0-9]+-[0-9]+-[0-9]+", string)
        Register_number.extend(d)
mylist = list(dict.fromkeys(Register_number))
print(len(mylist))

89


In [17]:
dataset = pd.DataFrame(mylist,columns =['CAS Number'])
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Not in list'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.head(2)

,CAS Number,value
0,2385-85-5,NaN
1,15972-60-8,NaN


In [18]:
result['Jurisdiction'] = 'Canada'
result['RegulatoryAgency'] = 'Canada Environmental Protection Agency (CEPA)'
result['TypeOfRegulation'] = 'Trade'
result['Criteria'] = 'Export banned or regulated'


result['Description'] = 'Export is banned or regulated'
result.loc[result['value'] == 'Not in list', 'Description'] = 'Export is not banned or regulated'

result['HazardClassification'] = 'High'
result.loc[result['value'] == 'Not in list', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = 'On the Export Control List'
result.loc[result['value'] == 'Not in list', 'CauseOfClassification'] = 'Not on Export COotrol List'

result['SourceName'] = 'Government of Canada'
result['SourceURL'] = 'https://www.canada.ca/en/environment-climate-change/services/canadian-environmental-protection-act-registry/substances-list/export-control-list.html'

result = result.drop(['value'], axis=1)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,2385-85-5,Canada,Canada Environmental Protection Agency (CEPA),Trade,Export banned or regulated,Export is banned or regulated,High,On the Export Control List,Government of Canada,https://www.canada.ca/en/environment-climate-c...
1,15972-60-8,Canada,Canada Environmental Protection Agency (CEPA),Trade,Export banned or regulated,Export is banned or regulated,High,On the Export Control List,Government of Canada,https://www.canada.ca/en/environment-climate-c...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [19]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:12:36.190327
